In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import glob
import pickle
import os
import csv
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.linear_model import LogisticRegression
from statistics import mean
warnings.filterwarnings('ignore')

In [48]:
sds_results_list = [f.name for f in os.scandir("../Adult_SDS/pickle/") if f.is_dir()==False]
sds_results = {}

In [49]:
original_dataset=pd.read_csv('../adultODS10K.csv', na_values=["?"])

In [50]:
features=[]

In [51]:
# generate random integer values
#from random import seed
from random import randint
# seed random number generator
#seed(1)
random_100 = []
random_1000 = []
# generate some integers
for _ in range(10):
    random_1000.append(randint(0, 999))
    random_100.append(randint(0, 99))
print(random_100)
print(random_1000)

[99, 0, 44, 67, 92, 59, 49, 26, 6, 4]
[791, 373, 870, 370, 50, 741, 95, 115, 699, 544]


In [52]:
def cross_tab(df, obs1=[]):
    observed=pd.crosstab(obs1,df['income'])
    val=stats.chi2_contingency(observed)
    return(val[1])

In [53]:
#prepare dataset and which feaures to use for classfication
def prepare_data(df, use_global_feature:bool):
    df_new=df.dropna(axis=0)
    #print(df_new.head())
    #print('Dimensions:',df_new.shape)
    col=df_new.columns
    data_type=df_new.dtypes
    uniq=df_new.nunique()

    #print("\n%30s  %10s   %10s\n " % ("Column Name", "Data Type", "Unique Values"))
    #for i in range(len(df_new.columns)):
        #print("%30s  %10s   %10s " % (col[i],data_type[i],uniq[i]))

    #print("\nDimensions:",df_new.shape[0],'rows and ',df_new.shape[1],'columns')
    df_new['income'].replace({'<=50K':0,'>50K':1},inplace=True)
    
    from collections import Counter
    occupatn=dict(Counter(df_new['occupation'])).keys()
    #print('Occupation types:','\n',list(occupatn),'\n')
    race=dict(Counter(df_new['race'])).keys()
    #print('Race types:','\n',list(race),'\n')
    relation=dict(Counter(df_new['relationship'])).keys()
    #print('Relation types:','\n',list(relation),'\n')
    educate=dict(Counter(df_new['education'])).keys()
    #print('Education levels:','\n',list(educate),'\n')
    marital=dict(Counter(df_new['marital.status'])).keys()
    #print('Marital status levels:','\n',list(marital),'\n')
    work=dict(Counter(df_new['workclass'])).keys()
    #print('Workclass levels:','\n',list(work),'\n')
    country=dict(Counter(df_new['native.country'])).keys()
    #print('Native countries:','\n',list(country),'\n')
    
    categorical_cols = df_new.columns[df_new.dtypes==object].tolist()
    categorical_cols
    
    pd.crosstab(df.relationship,df_new['income'])
    
    df_new_1=pd.get_dummies(df_new,columns=categorical_cols)
    df_new_1.head()
    
    from sklearn.preprocessing import MinMaxScaler
    columns_to_scale = ['age', 'capital.gain', 'capital.loss', 'hours.per.week']
    mms = MinMaxScaler()
    min_max_scaled_columns = mms.fit_transform(df_new_1[columns_to_scale])
    #processed_data = np.concatenate([min_max_scaled_columns, adult_new], axis=1)
    df_new_1['age'],df_new_1['capital.gain'],df_new_1['capital.loss'],df_new_1['hours.per.week']=min_max_scaled_columns[:,0],min_max_scaled_columns[:,1],min_max_scaled_columns[:,2],min_max_scaled_columns[:,3]
    df_new_1.head()
    
    
    if(use_global_feature == False):
        category=df_new_1.columns[df_new_1.dtypes!=object].tolist()[5:]
        #category
        alpha=0.01
        #df=adult_new.drop('income',axis=1)
        count=0
        for i in category:
            p_value=cross_tab(df_new, df_new_1[i])
            if p_value<=alpha:
                count+=1
                features.append(i)
                #print(i,' has a relation')
                #print('p-value for ',i,' is ',cross_tab(adult_new[i]),'\n')
        #print('Number of contributing attributes:',count,'\n')
        #print(features)

        features.append('age')
        features.append('capital.gain')
        features.append('capital.loss')
        features.append('hours.per.week')
        features.append('income')
  
    
    chi2=df_new_1[features]
    return chi2

In [54]:
#split dataset into train and testset
def train_print(clf,x_test,y_test):
    predictions = clf.predict(x_test)
    print('Precision report:\nprecision \t\t\t recall \t\t\t f-score \t\t\t support\n',
          precision_recall_fscore_support(y_test, predictions)[0],'\t',
          precision_recall_fscore_support(y_test, predictions)[1],
          '\t',precision_recall_fscore_support(y_test, predictions)[2],'\t',
          precision_recall_fscore_support(y_test, predictions)[3],'\n')
    print('Confusion matrix:\n',confusion_matrix(y_test, predictions),'\n')
    print('Accuracy score:',accuracy_score(y_test, predictions)*100,'\n')

In [146]:
# Train models and test accuracy
accuracy_list=[]
accuracy_list_only_synthetic = []
accuracy_list_original= []
#logmodel = [[None]*(len(random_1000))]*(len(sds_results_list))
logmodel = np.empty((len(sds_results_list), len(random_1000)), object)
logmodel_original = np.empty(17, object)


sds_results_df = pd.DataFrame(index = [[i.split('.')[0] for i in sds_results_list]], columns=['average accuracy(trS-teO)', 'average accuracy(trS-teS)', 'accuracy original'])

#-------------------------------Train and Test on Original dataset----------------------------------------#

features.clear()
chi2_original = prepare_data(original_dataset, False)

x_original = chi2_original.drop('income', axis=1)
y_original = chi2_original['income']

for i in range(17):
    x_train_original, x_test_original, y_train_original, y_test_original = \
                train_test_split(x_original, y_original, test_size=0.20, random_state=np.random , shuffle =True)
    X_train_original, Y_train_original = SMOTE().fit_resample(x_train_original, y_train_original)

    logmodel_original[i] = LogisticRegression()
    logmodel_original[i].fit(X_train_original, Y_train_original)

    #train_print(logmodel,x_test,y_test)

    predictions_original = logmodel_original[i].predict(x_test_original)
    accuracy_list_original.append(accuracy_score(y_test_original, predictions_original)*100)
    sds_results_df.at[sds_results_list[i].split('.')[0], 'accuracy original'] = accuracy_score(y_test_original, predictions_original)*100
print(accuracy_list_original)

#---------------------------------------------------End----------------------------------------------------#
row = -1
for method in sds_results_list:
    df=[]
    row += 1
    accuracy_list.clear()
    accuracy_list_only_synthetic.clear()
    print('Starting file '+ method)
    with open('../Adult_SDS/pickle/'+ method, 'rb') as input_file:
        df = pickle.load(input_file)
    random_index = random_1000
    if(len(df) == 100):
        random_index = random_100
    
    col = -1
    for i in random_index:
        #features.clear()
        col += 1
        chi2 = prepare_data(df[i], True)
        test_set=prepare_data(original_dataset, True)
        
#-------------------------Train on Synthetic and Test on Original----------------------------------------#
        #x = chi2.drop('income', axis=1)
        #y = chi2['income']
        x_train = chi2.drop('income', axis=1)
        y_train = chi2['income']
        x_test = test_set.drop('income', axis=1)
        y_test = test_set['income']
        #x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1)
        X_train, Y_train = SMOTE().fit_resample(x_train, y_train)
        
        logmodel[row][col] = LogisticRegression()
        logmodel[row][col].fit(X_train, Y_train)

        #train_print(logmodel,x_test,y_test)

        predictions = logmodel[row][col].predict(x_test)
        accuracy_list.append(accuracy_score(y_test, predictions)*100)
#---------------------------------------------------End----------------------------------------------------#
        
#-------------------------------Train and Test on Synthetic dataset----------------------------------------#
        x_only_synthetic = chi2.drop('income', axis=1)
        y_only_synthetic = chi2['income']
        x_train_only_synthetic, x_test_only_synthetic, y_train_only_synthetic,\
                    y_test_only_synthetic = train_test_split(x_only_synthetic, y_only_synthetic, \
                    test_size=0.20, random_state=1)
        X_train_only_synthetic, Y_train_only_synthetic = SMOTE().fit_resample(x_train_only_synthetic, y_train_only_synthetic)
        
        logmodel_only_synthetic = LogisticRegression()
        logmodel_only_synthetic.fit(X_train_only_synthetic, Y_train_only_synthetic)

        #train_print(logmodel,x_test,y_test)

        predictions_only_synthetic = logmodel_only_synthetic.predict(x_test_only_synthetic)
        accuracy_list_only_synthetic.append(accuracy_score(y_test_only_synthetic, predictions_only_synthetic)*100)
#---------------------------------------------------End----------------------------------------------------#    
    print('trs-teO ', accuracy_list)
    print('trS-teS ',accuracy_list_only_synthetic)
    sds_results_df.at[method.split('.')[0], 'average accuracy(trS-teO)'] = mean(accuracy_list)
    sds_results_df.at[method.split('.')[0], 'average accuracy(trS-teS)'] = mean(accuracy_list_only_synthetic)
    
    # Write results to CSV
    #with open('Test_Results/Adult_individual_classification_accuracy.csv', 'a+', newline='') as f: 
        #write = csv.writer(f) 
        #write.writerow([method.split('.')[0], 'trS-teO', accuracy_list, 'trS-teS', accuracy_list_only_synthetic]) 
        #write.writerow(accuracy_list) 
        #new_list.append([method.split('.')[0], accuracy_list])
    #print(method.split('.')[0] + ' = ', sds_results[method.split('.')[0]])
        #print(new_list)
print(sds_results_df)

[79.45, 79.35, 78.25, 77.60000000000001, 79.95, 79.10000000000001, 79.60000000000001, 78.60000000000001, 79.05, 79.75, 80.0, 80.7, 80.0, 81.2, 78.45, 80.85, 79.55]
Starting file sdsAdultCatMC.pkl
trs-teO  [79.4, 79.59, 79.52, 79.67, 79.84, 79.24, 79.97, 79.58, 79.45, 79.86]
trS-teS  [78.2, 81.69999999999999, 80.75, 78.55, 79.14999999999999, 77.64999999999999, 81.10000000000001, 78.2, 80.2, 81.2]
Starting file sdsAdultCatMCT.pkl
trs-teO  [79.32000000000001, 79.74, 78.8, 78.82000000000001, 79.52, 79.19, 80.08999999999999, 79.67999999999999, 79.42, 79.60000000000001]
trS-teS  [79.25, 79.0, 79.85, 81.3, 80.45, 80.60000000000001, 81.39999999999999, 80.7, 79.35, 80.2]
Starting file sdsAdultCatMP.pkl
trs-teO  [78.91, 79.31, 79.17999999999999, 78.86, 79.11, 79.17999999999999, 79.46, 79.55, 78.84, 79.03999999999999]
trS-teS  [79.0, 80.75, 79.25, 78.85, 79.9, 78.45, 80.45, 79.0, 78.25, 80.25]
Starting file sdsAdultCatMPT.pkl
trs-teO  [79.13, 79.12, 79.35, 79.03999999999999, 78.86999999999999, 78

In [156]:
mean(accuracy_list_original)

79.49705882352941

In [147]:
# Testset from original adult datset reconrd number 10000-15000
original_testset=pd.read_csv('../adultODS10K_to_15K.csv', na_values=["?"])

In [148]:
column_names = ['Original']
column_names.extend(sds_results_list)
column_names = [s.replace(".pkl", "") for s in column_names]
df_wrong_classfication = pd.DataFrame(index = [[i for i in range(1, (len(random_1000)+1))]], columns = column_names)
df_wrong_classfication

,Original,sdsAdultCatMC,sdsAdultCatMCT,sdsAdultCatMP,sdsAdultCatMPT,sdsAdultDecM,sdsAdultDecMH,sdsAdultDecMHT,sdsAdultDecML,sdsAdultDecMLT,sdsAdultDecMT,sdsAdultParM,sdsAdultParMH,sdsAdultParMHT,sdsAdultParML,sdsAdultParMLT,sdsAdultParMT,sdsAdultSamp
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [155]:
# check which records are incorrectly classified by original model
wrong_list = []
chi2_original = prepare_data(original_testset, True)
x = chi2_original.drop('income', axis=1)
y = chi2_original['income']

for i in range(0, 10):
    wrong_list.clear()
    pre = logmodel_original[i].predict(x)
    for row_index, (input, prediction, label) in enumerate(zip (x, pre, y)):
        if prediction != label:
            wrong_list.append(row_index)
            #print('Row', row_index, 'has been classified as ', prediction, 'and should be ', label)
    print(wrong_list)
    df_wrong_classfication.iat[i, df_wrong_classfication.columns.get_loc('Original')] = wrong_list   

[1, 3, 6, 12, 13, 20, 23, 32, 37, 43, 46]
[1, 3, 6, 12, 13, 20, 23, 32, 33, 37, 43, 46]
[1, 3, 6, 12, 13, 20, 23, 32, 37, 43, 46]
[1, 3, 6, 12, 13, 20, 23, 32, 37, 43, 46]
[1, 3, 6, 12, 13, 20, 23, 33, 37, 43, 46]
[1, 3, 6, 12, 13, 20, 23, 37, 43, 46]
[1, 3, 6, 12, 13, 20, 23, 32, 37, 43, 46]
[1, 3, 6, 12, 13, 20, 23, 32, 33, 37, 43, 46]
[1, 3, 6, 12, 13, 20, 23, 32, 37, 43, 46]
[1, 3, 6, 12, 13, 20, 23, 32, 37, 43, 46]


In [151]:
# check which records are incorrectly classified by synthetic model
wrong_list = []
chi2_synthetic = prepare_data(original_testset, True)
x = chi2_synthetic.drop('income', axis=1)
y = chi2_synthetic['income']
for i in range(0, len(logmodel)):
    print(sds_results_list[i].split('.')[0])
    for j in range(0, len(logmodel[i])):
        wrong_list.clear()
        pre = logmodel[i][j].predict(x)
        for row_index, (input, prediction, label) in enumerate(zip (x, pre, y)):
            if prediction != label:
                wrong_list.append(row_index)
                #print('Row', row_index, 'has been classified as ', prediction, 'and should be ', label)
        print(wrong_list)
        df_wrong_classfication.iat[j, df_wrong_classfication.columns.get_loc(sds_results_list[i].split('.')[0])] = wrong_list

sdsAdultCatMC
[1, 3, 6, 12, 13, 20, 23, 32, 37, 43, 46, 50]
[1, 3, 6, 12, 20, 23, 32, 37, 43, 46]
[1, 3, 6, 12, 20, 23, 32, 37, 43, 46]
[1, 3, 6, 12, 13, 20, 23, 32, 33, 43, 46]
[1, 3, 6, 12, 13, 20, 23, 33, 37, 43, 46]
[1, 3, 6, 12, 20, 23, 32, 37, 43, 46]
[1, 3, 6, 12, 20, 23, 32, 37, 43, 46]
[1, 3, 6, 12, 13, 20, 23, 32, 37, 43, 46]
[1, 3, 6, 12, 13, 20, 23, 32, 37, 43, 46, 50]
[1, 3, 6, 12, 13, 20, 23, 37, 43, 46, 50]
sdsAdultCatMCT
[1, 3, 6, 12, 20, 23, 32, 37, 43, 46]
[1, 3, 6, 12, 13, 20, 23, 32, 37, 43, 46]
[1, 3, 6, 12, 20, 23, 32, 33, 43, 46]
[1, 3, 6, 12, 20, 23, 32, 33, 43, 46, 50]
[1, 3, 6, 12, 20, 23, 32, 37, 43, 46]
[1, 3, 6, 12, 13, 20, 23, 37, 43, 46, 50]
[1, 3, 6, 12, 20, 23, 32, 33, 37, 43, 46, 50]
[1, 3, 6, 12, 20, 23, 33, 37, 43, 46]
[1, 3, 6, 12, 13, 20, 23, 32, 33, 37, 43, 46, 50]
[1, 3, 6, 12, 13, 20, 23, 32, 33, 37, 43, 46, 50]
sdsAdultCatMP
[1, 3, 6, 12, 13, 20, 23, 32, 37, 42, 43, 46, 50]
[1, 3, 6, 12, 13, 20, 23, 32, 37, 43, 46]
[1, 3, 6, 12, 13, 20, 23, 32,

In [152]:
df_wrong_classfication

,Original,sdsAdultCatMC,sdsAdultCatMCT,sdsAdultCatMP,sdsAdultCatMPT,sdsAdultDecM,sdsAdultDecMH,sdsAdultDecMHT,sdsAdultDecML,sdsAdultDecMLT,sdsAdultDecMT,sdsAdultParM,sdsAdultParMH,sdsAdultParMHT,sdsAdultParML,sdsAdultParMLT,sdsAdultParMT,sdsAdultSamp
1,"[1, 3, 6, 12, 13, 20, 23, 32, 37, 43, 46]","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37..."
2,"[1, 3, 6, 12, 13, 20, 23, 32, 37, 43, 46]","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37..."
3,"[1, 3, 6, 12, 13, 20, 23, 32, 37, 43, 46]","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37..."
4,"[1, 3, 6, 12, 13, 20, 23, 32, 37, 43, 46]","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37...","[1, 2, 3, 6, 9, 12, 13, 16, 18, 25, 27, 33, 37..."
5,"[1, 3, 6, 12, 13, 20, 23, 